In [2]:
import os.path as osp
import numpy as np
def make_filepath_list(rootpath):
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'Annotations', '%s.xml')

    train_id_names = osp.join(rootpath, 'ImageSets', 'Main', 'train.txt')
    val_id_names = osp.join(rootpath, 'ImageSets', 'Main', 'val.txt')

    train_id_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_id_list.append(img_path)
        train_anno_list.append(anno_path)
    
    val_id_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_id_list.append(img_path)
        val_anno_list.append(anno_path)
    return train_id_list, train_anno_list, val_id_list, val_anno_list

In [3]:
rootpath = './data/VOCdevkit/VOC2012'
tr_img, tr_anno, val_img, val_anno = make_filepath_list(rootpath)
print(tr_img[0])
print(tr_anno[0])
print(val_img[0])
print(val_anno[0])

./data/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg
./data/VOCdevkit/VOC2012/Annotations/2008_000008.xml
./data/VOCdevkit/VOC2012/JPEGImages/2008_000002.jpg
./data/VOCdevkit/VOC2012/Annotations/2008_000002.xml


In [6]:
import xml.etree.ElementTree as ElementTree
import numpy as np
class GetBBoxAndLabel(object):
    def __init__(self,classes):
        self.classes = classes
    def __call__(self, xml_path, width,height):
        annotaion = []
        xml = ElementTree.parse(xml_path).getroot()
        for obj in xml.iter('object'):
            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                continue
            bndbox = []
            name = obj.find('name').text.lower().strip()
            bbox = obj.find('bndbox')
            grid = ['xmin', 'ymin', 'xmax', 'ymax']
            for gr in (grid):
                axis_value = int(bbox.find(gr).text)
                if gr == 'xmin' or gr == 'xmax':
                    axis_value = axis_value / width
                else:
                    axis_value = axis_value / height
                bndbox.append(axis_value)
            label_idx = self.classes.index(name)
            bndbox.append(label_idx)
            annotaion.append(bndbox)
        return np.array(annotaion)

In [19]:
import cv2
voc_labels = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
    'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
    'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]
annotaion = GetBBoxAndLabel(voc_labels)
index = 1
img_path = val_img[index]
img = cv2.imread(img_path)
height, width, _ = img.shape
annotaion(val_anno[index], width, height)

array([[ 0.092     ,  0.03303303,  1.        ,  1.        , 18.        ],
       [ 0.124     ,  0.57057057,  0.166     ,  0.72972973, 14.        ]])